<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# AutoGen Routing Agent

In this tutorial, we'll explore Routing agent workflows using [AutoGen's function calling capabilities and agent interactions](https://microsoft.github.io/autogen-for-net/articles/Create-an-agent.html) to direct tasks efficiently.

Agent Routing is a pattern designed to handle incoming requests by classifying them and directing them to the single most appropriate specialized agent or workflow. This ensures requests are immediately handled by the correct specialist, improving efficiency.

AutoGen simplifies implementing this pattern by enabling a dedicated 'Router Agent' to analyze incoming messages and use function calls to signal its classification decision. Based on this classification, the workflow explicitly directs the query to the appropriate specialist agent for a focused, separate interaction.

With Phoenix tracing, you get full visibility into this process, clearly seeing the router's classification, the function call made, and the subsequent handoff to the specialist.

By the end of this tutorial, you’ll learn how to:

- Define AutoGen agents with specific tools (functions).
- Create a dedicated Router Agent for request classification.
- Implement a workflow that routes requests to specialist agents based on classification.
- Trace and visualize the routing process and agent interactions using Phoenix.

⚠️ You'll need an OpenAI Key for this tutorial.

## Set up Keys and Dependencies


In [ ]:
!pip install -qqq arize-phoenix arize-phoenix-otel openinference-instrumentation-openai

In [ ]:
!pip install -qq pyautogen==0.9 autogen-agentchat~=0.2

In [ ]:
import os
from getpass import getpass

import autogen

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

## Configure Tracing


In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="autogen-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

## Example Routing Task: Customer Support Agent

We will build an intelligent customer service system, designed to efficiently handle diverse user queries by leveraging specialized agents. We use tracing to tackle the common challenge of ensuring incoming requests are promptly routed to the appropriate agent.

Below is a high-level diagram of our setup. Specifically, a dedicated router agent analyzes the user's initial message solely to classify its topic, triggering a handoff. A specialized agent then uses its tools to address the query. Once a termination condition is reached, the user gets an output.

![Diagram](https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_routing_diagram.png)



## Define Agent

The `llm_config` specifies the configuration used for each `AssistantAgent`.


In [ ]:
config_list = {
    "model": "gpt-4o",
    "api_key": os.environ["OPENAI_API_KEY"],
}
llm_config = {"config_list": config_list}

This cell defines functions that simulate typical operations for a customer support scenario, such as retrieving user subscription data, checking payment history, verifying server status, and handling account actions like password resets. In a real setting, we would use data retrieval techniques.

These functions will act as the tools for our agents.

In [ ]:
# Function Tools
import json


def lookup_subscription(user_id: str):
    print(f"lookup_subscription for user: {user_id}")
    subscriptions = {
        "user123": {"plan": "Pro", "status": "Active", "renewal_date": "2025-12-01"},
        "user456": {"plan": "Free", "status": "Active", "renewal_date": None},
    }
    data = subscriptions.get(user_id)
    return json.dumps(data)


def check_payment_history(user_id: str, limit: int = 3):
    print(f"check_payment_history for user: {user_id}")
    history = {
        "user123": [
            {"date": "2025-04-01", "amount": 20.00, "status": "Paid"},
            {"date": "2025-03-01", "amount": 20.00, "status": "Paid"},
        ],
        "user456": [],
    }
    data = history.get(user_id, [])
    return json.dumps(data[:limit])


def check_server_status(service_name: str):
    status = {
        "login_service": "Operational",
        "payment_gateway": "Operational",
        "app_server": "Not Operational",
    }
    data = {"service": service_name, "status": status.get(service_name, "Unknown")}
    return json.dumps(data)


def request_password_reset(user_id: str):
    print(f"request_password_reset for user: {user_id}")
    success = user_id in ["user123", "user456"]
    data = {"user_id": user_id, "reset_initiated": success}
    return json.dumps(data)


def check_account_status(user_id: str):
    status = {
        "user123": "Active",
        "user456": "Active",
        "user789": "Inactive",
    }
    data = {"user_id": user_id, "status": status.get(user_id, "Not Found")}
    return json.dumps(data)

Next, we define three AutoGen `AssistantAgent` instances, each specialized for a specific customer support function: Billing, Technical support, and Account Management.

Each agent is given a unique system message describing its role, and its `llm_config` includes schemas for only the specific function-calling tools relevant to its domain.

In [ ]:
# Billing Agent
billing_agent = autogen.AssistantAgent(
    name="Billing_Support_Agent",
    system_message="You are a billing support specialist. You can help with subscriptions, payments, and refunds. Use your available tools.",
    llm_config={
        "config_list": config_list,
        "functions": [
            {
                "name": "lookup_subscription",
                "description": "Get user subscription details.",
                "parameters": {
                    "type": "object",
                    "properties": {"user_id": {"type": "string", "description": "The user ID"}},
                    "required": ["user_id"],
                },
            },
            {
                "name": "check_payment_history",
                "description": "Check recent payment history.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user_id": {"type": "string", "description": "The user ID"},
                        "limit": {"type": "integer", "description": "Max number of records"},
                    },
                    "required": ["user_id"],
                },
            },
        ],
    },
)

# Technical Agent
technical_agent = autogen.AssistantAgent(
    name="Technical_Support_Agent",
    system_message="You are a technical support specialist. You can help troubleshoot issues, check service status, and search the knowledge base. Use your available tools.",
    llm_config={
        "config_list": config_list,
        "functions": [
            {
                "name": "check_server_status",
                "description": "Check the status of a backend service.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "service_name": {
                            "type": "string",
                            "description": "Name of the service (e.g., login_service)",
                        }
                    },
                    "required": ["service_name"],
                },
            },
        ],
    },
)

# Account Management Agent
account_agent = autogen.AssistantAgent(
    name="Account_Management_Agent",
    system_message="You are an account management specialist. You can help with account status and password resets. Use your available tools.",
    llm_config={
        "config_list": config_list,
        "functions": [
            {
                "name": "request_password_reset",
                "description": "Initiate a password reset for the user.",
                "parameters": {
                    "type": "object",
                    "properties": {"user_id": {"type": "string", "description": "The user ID"}},
                    "required": ["user_id"],
                },
            },
            {
                "name": "check_account_status",
                "description": "Check if a user account is active.",
                "parameters": {
                    "type": "object",
                    "properties": {"user_id": {"type": "string", "description": "The user ID"}},
                    "required": ["user_id"],
                },
            },
        ],
    },
)

This section defines two helper functions:
- `route_to_specialist` gets the topic classification decision made by a router LLM into a global variable
- `check_for_termination_message` identifies if a message signals the end of the chat.

Then, we initialize the main AutoGen `UserProxyAgent` and configuring it to use these functions for termination checks and equipping it to execute any of the tool functions when called by an `AssisstantAgent`. The `UserProxyAgent` also acts as the human user within the AutoGen framework, initiating conversations and managing user interaction.

In [ ]:
def route_to_specialist(topic: str, user_query_for_specialist: str):
    print(f"Router determined topic: {topic}")
    global routing_decision
    routing_decision = {"topic": topic, "query": user_query_for_specialist}

    if topic in ["billing", "technical", "account"]:
        return f"Routing request about '{topic}'"
    else:
        return "Could not determine a category."


def check_for_termination_message(message_dict):
    content = message_dict.get("content")
    if isinstance(content, str):
        return content.rstrip().endswith("exit")
    return False


user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    system_message="A human user. You will execute functions when requested.",
    human_input_mode="TERMINATE",
    is_termination_msg=check_for_termination_message,
    function_map={
        "lookup_subscription": lookup_subscription,
        "check_payment_history": check_payment_history,
        "check_server_status": check_server_status,
        "request_password_reset": request_password_reset,
        "check_account_status": check_account_status,
        "route_to_specialist": route_to_specialist,
    },
)

Finally, we define our router. This `AssistantAgent` classifies incoming customer support requests into categories: billing, technical, account, or unknown.

The system message instructs the `router_agent` not to answer queries directly, but instead to analyze the request and call the specific `route_to_specialist` function.

In [ ]:
router_agent = autogen.AssistantAgent(
    name="Support_Router",
    system_message="""You are a customer support request router. Your job is to analyze the user's request and determine the primary topic: 'billing', 'technical', or 'account'.
    Based on the primary topic, call the 'route_to_specialist' function with the identified topic and the user query.
    Do NOT attempt to answer the user's query yourself. Only classify and call the function.
    If the query is ambiguous, very general, or doesn't fit these categories, use the topic 'unknown'.
    """,
    llm_config={
        "config_list": config_list,
        "temperature": 0,
        "functions": [
            {
                "name": "route_to_specialist",
                "description": "Route the user's request to the appropriate specialist based on the primary topic.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "topic": {
                            "type": "string",
                            "description": "The topic identified ('billing', 'technical', 'account', or 'unknown').",
                        },
                        "user_query_for_specialist": {
                            "type": "string",
                            "description": "The original user query to be passed to the specialist.",
                        },
                    },
                    "required": ["topic", "user_query_for_specialist"],
                },
            }
        ],
    },
)

## Run Agent

We are now ready to run our agent. First, the user will be prompted for their query. Then there is an interaction between the `UserProxyAgent` and the `router_agent` to classify the request's topic.

Based on the classification result captured from the first interaction, it either initiates a separate, follow-up chat with the appropriate `AssisstantAgent` or provides feedback to the user if routing was unsuccessful.

In [ ]:
routing_decision = None
specialist_map = {
    "billing": billing_agent,
    "technical": technical_agent,
    "account": account_agent,
}

initial_query = input("Please enter your customer support question: ")

tracer = tracer_provider.get_tracer(__name__)
with tracer.start_as_current_span(
    "CustomerSupportInteraction",
    openinference_span_kind="agent",
) as agent_span:
    user_proxy.initiate_chat(
        router_agent,
        message=initial_query,
        max_turns=2,
    )

    print(f"Routing decision: {routing_decision}")

    selected_specialist = None
    query_for_specialist = None

    if routing_decision and routing_decision["topic"] in specialist_map:
        selected_specialist = specialist_map[routing_decision["topic"]]
        query_for_specialist = routing_decision["query"]
        print(f"Routing to: {selected_specialist.name}")

        user_proxy.initiate_chat(
            selected_specialist,
            message=f"Hello {selected_specialist.name}, I need help with the following: {query_for_specialist}",
            max_turns=5,
        )

    elif routing_decision:
        print(
            f"Routing determined topic: {routing_decision['topic']}. No specific specialist assigned."
        )
        print(
            "Can you please specify if your issue relates to billing, technical problems, or account management?"
        )
    else:
        print("Routing failed.")

# Example Queries To Test:
# initial_query = "Hi, I think I was double charged last month for user user123."
# initial_query = "The login page isn't loading, is the login_service down?"
# initial_query = "I forgot my password for user456, can you help?"

## View Results in Phoenix

In Phoenix, we unlock observability into our agent's operations and decision-making process. The traces allow easy inspection of foundational elements like system prompts and function call. Phoenix also showcases the reasoning behind routing choices and provides transparency into tool interactions by tracking both input parameters and generated outputs.

Run the cell below to see the full tracing results.



In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/arize-phoenix-assets/assets/gifs/autogen_router_results.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")